In [1]:
%pip install transformers datasets evaluate accelerate peft tf-keras


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


# RoBERTa Base

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset, concatenate_datasets
import torch
import random
import numpy as np
from peft import LoraModel, LoraConfig
from evaluate import load
import math

if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)

mps


In [11]:
seed = 0
def set_seed(seed: int):
    # Set seed for Python's random module
    random.seed(seed)

    # Set seed for NumPy
    np.random.seed(seed)

    # Set seed for PyTorch
    torch.manual_seed(seed)

    # Set seed for CUDA (if using)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # For multi-GPU setups

    # Make PyTorch deterministic (this can slow down the computation)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Example of usage
set_seed(seed)


In [7]:
dataset = load_dataset("glue", "qqp")

Generating train split:   0%|          | 0/363846 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/40430 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/390965 [00:00<?, ? examples/s]

In [8]:
dataset['train']

Dataset({
    features: ['question1', 'question2', 'label', 'idx'],
    num_rows: 363846
})

### Test set contains unlabeled data (-1). Hence, we will need to redo the trainig/evaluation/test split

In [9]:
print(f"Labels in the training set: {set(dataset['train']['label'])}")
print(f"Labels in the evaluation set: {set(dataset['validation']['label'])}")
print(f"Labels in the test set: {set(dataset['test']['label'])}")

Labels in the training set: {0, 1}
Labels in the evaluation set: {0, 1}
Labels in the test set: {-1}


### Combine validation / test sets

In [12]:
#combined_test_set = concatenate_datasets([dataset['test_matched'], dataset['test_mismatched']])

merged_dataset = concatenate_datasets([dataset['validation'], dataset['train']])

# Split into 80% train and 20% temp (for test + validation)
temp_split = merged_dataset.train_test_split(test_size=0.2, seed=seed)
train_dataset = temp_split["train"]
temp_dataset = temp_split["test"]

# Further split temp into 50% test and 50% validation
test_eval_split = temp_dataset.train_test_split(test_size=0.5, seed=seed)
test_dataset = test_eval_split["train"]
eval_dataset = test_eval_split["test"]


print(f"Labels in the test set after processing: {set(test_dataset['label'])}")

Labels in the test set after processing: {0, 1}


### Split: 80% Training data / 10% Evaluation / 10% Test

In [13]:
# Load Roberta Base
model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
num_labels = len(set(dataset['train']['label']))
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
print(num_labels)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


2


In [26]:
parameters = {
    "output_folder": 'qqp',
    "sequence_length": 64,
    "epochs": 1,
    "batch_size": 16,
    "learning_rate": 5e-4,
    "weight_decay": 0.00,
    "warmup_ratio": 0.06,
    "optimizer": 'adamw_torch',
    "lora_alpha": 8,
    "lora_rank": 8,
    "lora_target_modules": ["query", "key", "value"],
    "lora_drop_out": 0.0
}


In [27]:
# Load "lora-fied" model
target_modules = ["query", "key", "value"]
config = LoraConfig(
    task_type="SEQ_CLS",
    r=parameters["lora_rank"],
    lora_alpha=parameters["lora_alpha"],
    target_modules=parameters["lora_target_modules"],
    lora_dropout=parameters["lora_drop_out"],
    init_lora_weights=True
)
#lora_model = LoraModel(model, config, "default")

from peft import  get_peft_model
peft_model = get_peft_model(model, config)

### We report the overall (matched and mismatched) accuracy for MNLI, Matthew’s correlation for CoLA, Pearson correlation for STS-B, and accuracy for other tasks.

In [21]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question1', 'question2', 'label', 'idx'],
        num_rows: 363846
    })
    validation: Dataset({
        features: ['question1', 'question2', 'label', 'idx'],
        num_rows: 40430
    })
    test: Dataset({
        features: ['question1', 'question2', 'label', 'idx'],
        num_rows: 390965
    })
})

In [17]:
dataset['validation'].data

MemoryMappedTable
question1: string
question2: string
label: int64
idx: int32
----
question1: [["Why are African-Americans so beautiful?","I want to pursue PhD in Computer Science about social network,what is the open problem in social networks?","Is there a reason why we should travel alone?","Why are people so obsessed with having a girlfriend/boyfriend?","What are some good baby girl names starting with D?",...,"Which is the smartest city in the world?","What is the corporate culture like at Owens Corning? How is the culture different than other companies?","which laptop will be best under Rs 30,000?","What are some things new employees should know going into their first day at Metals USA?","How long can I keep a BigMac in my fridge before eating it?"],["What are your views on the Brexit?","Whether a Law graduate with PhD in science and having PG Diploma in IPR is required to write a Patent Agent Exam?","What are some funny stories in engineering?","What are the best books, topics, 

In [28]:
# Tokenize the dataset
def preprocess_function(examples):
    return tokenizer(examples["question1"], examples['question2'],truncation=True, max_length=parameters['sequence_length'])

# Prepare datasets for training

# For debug purposes: Load only 1 % of the dataset
#train_dataset_tokenized = dataset['train'].map(preprocess_function, batched=True).shuffle(seed=seed)
#eval_dataset_tokenized = dataset['validation'].map(preprocess_function, batched=True).shuffle(seed=seed)
#test_dataset_tokenized = dataset['test'].map(preprocess_function, batched=True)


train_dataset_tokenized = train_dataset.map(preprocess_function, batched=True).shuffle(seed=seed)
eval_dataset_tokenized = eval_dataset.map(preprocess_function, batched=True).shuffle(seed=seed)
test_dataset_tokenized = test_dataset.map(preprocess_function, batched=True)


Map:   0%|          | 0/323420 [00:00<?, ? examples/s]

Map:   0%|          | 0/40428 [00:00<?, ? examples/s]

Map:   0%|          | 0/40428 [00:00<?, ? examples/s]

In [29]:
# Define training arguments
training_args = TrainingArguments(
    output_dir=f"./{parameters['output_folder']}/results",  # Directory to save model checkpoints
    evaluation_strategy="epoch",
    learning_rate=parameters['learning_rate'],
    per_device_train_batch_size=parameters['batch_size'],
    per_device_eval_batch_size=parameters['batch_size'],
    num_train_epochs=parameters['epochs'],
    weight_decay=parameters['weight_decay'],
    save_strategy="epoch",
    logging_dir=f"./{parameters['output_folder']}/logs",
    logging_steps=10,
    optim=parameters['optimizer'],
    load_best_model_at_end=True,
    report_to="none",
    warmup_ratio=parameters['warmup_ratio'],
)

# Accuracy metric
metric = load('accuracy')

# Matthews correlation metric
#metric = load("matthews_correlation")

def compute_metric(p):
    preds = p.predictions.argmax(axis=1)
    labels = p.label_ids
    return metric.compute(predictions=preds, references=labels)

# Define the trainer
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_dataset_tokenized,
    eval_dataset=eval_dataset_tokenized,
    tokenizer=tokenizer,
    compute_metrics=compute_metric
)

# Train the model
#trainer.train(resume_from_checkpoint=True)
trainer.train()


# Save model
trainer.save_model(f"./{parameters['output_folder']}/model")

/var/folders/95/cqsd5hvd7xl6lhsmdv_177s80000gn/T/ipykernel_84513/160210836.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


  0%|          | 0/20214 [00:00<?, ?it/s]

{'loss': 0.6829, 'grad_norm': 1.2487291097640991, 'learning_rate': 4.122011541632317e-06, 'epoch': 0.0}
{'loss': 0.6814, 'grad_norm': 1.4245132207870483, 'learning_rate': 8.244023083264634e-06, 'epoch': 0.0}
{'loss': 0.6786, 'grad_norm': 0.7511841058731079, 'learning_rate': 1.236603462489695e-05, 'epoch': 0.0}
{'loss': 0.663, 'grad_norm': 2.5350494384765625, 'learning_rate': 1.6488046166529267e-05, 'epoch': 0.0}
{'loss': 0.6941, 'grad_norm': 0.941170871257782, 'learning_rate': 2.0610057708161584e-05, 'epoch': 0.0}
{'loss': 0.6592, 'grad_norm': 0.9832479357719421, 'learning_rate': 2.47320692497939e-05, 'epoch': 0.0}
{'loss': 0.6761, 'grad_norm': 2.318202018737793, 'learning_rate': 2.8854080791426218e-05, 'epoch': 0.0}
{'loss': 0.6507, 'grad_norm': 0.7846818566322327, 'learning_rate': 3.2976092333058534e-05, 'epoch': 0.0}
{'loss': 0.6649, 'grad_norm': 1.4472655057907104, 'learning_rate': 3.709810387469085e-05, 'epoch': 0.0}
{'loss': 0.6456, 'grad_norm': 0.801546573638916, 'learning_rate'

KeyboardInterrupt: 

In [28]:
# Evaluate model
results = trainer.evaluate()
print(results)

  0%|          | 0/66 [00:00<?, ?it/s]

{'eval_loss': 0.45087283849716187, 'eval_matthews_correlation': 0.5293023756875906, 'eval_runtime': 13.4779, 'eval_samples_per_second': 77.386, 'eval_steps_per_second': 4.897, 'epoch': 1.0}


In [29]:
# Test set
test_set_results = trainer.predict(test_dataset_tokenized)
test_set_results.metrics

  0%|          | 0/67 [00:00<?, ?it/s]

{'test_loss': 0.0,
 'test_matthews_correlation': 0.0,
 'test_runtime': 15.2682,
 'test_samples_per_second': 69.622,
 'test_steps_per_second': 4.388}

In [ ]:
test_set_results.predictions.argmax(axis=1)

array([1, 1, 1, ..., 1, 1, 1])

In [ ]:
test_set_results.label_ids

array([2, 0, 1, ..., 2, 2, 0])